In [1]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# modify html to text and read
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source)
post_table = soup.find("table", {"class": "wikitable"})

headers = {0: 'Postcode', 1: 'Borough', 2: 'Neighbourhood'}

rows = post_table.find_all('tr')

table_data = []

#first row include head of table, create our table
for count in range(1,len(rows)):
    cells = rows[count].find_all("td")
    item = {}
    for index in headers:
        item[headers[index]] = cells[index].text
    table_data.append(item)   

#table - to dataframe
df = pd.DataFrame(table_data)
df.head()
df.shape

(287, 3)

In [3]:
# delete rows where Borough  = 'Not assigned'
df = df.loc[df['Borough'] != 'Not assigned']

# delete symbols \n in Neighbourhood column
df['Neighbourhood'] = df['Neighbourhood'].str.replace('\n','')

# grouped all rows and aggregate Neighbourhood separated by ','
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].agg(', '.join).reset_index()

# replace 'Not assigned' in Neighbourhood on Borough
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']      
df.shape

(103, 3)